In [1]:
import os
import json
from hydra import compose, initialize
from omegaconf import OmegaConf, DictConfig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from vbf_tagger.tools.evaluation import general as g

with initialize(version_base=None, config_path="../config", job_name="test_app"):
    cfg = compose(config_name="main")

In [2]:
from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="../config", job_name="test_app"):
    cfg = compose(config_name="main")

print(OmegaConf.to_yaml(cfg))


host:
  name: manivald
  project_dir: /home/norman/vbf-tagger
  data_dir: /home/norman/vbf-tagger/vbf_tagger/data
  tmp_dir: /home/norman/tmp
  slurm:
    queue:
      preprocessing:
        output_dir: /home/norman/VBF_preprocessing
        partition: main
        time: 06:00:00
        cpus: 1
        mem_map:
          CEPC:
            one_step: 25G
            two_step: 25G
          FCC:
            one_step: 12G
            two_step: 12G
        mem: ${host.slurm.queue.preprocessing.mem_map[${dataset.name}][${preprocessing.data_type}]}
dataset:
  name: 22pre
  input_dim: 1200
  branches: null
  tree_path: null
  raw_input_dir: ${host.data_dir}/22pre/
  data_dir: ${host.data_dir}/22pre/hh_vbf/preprocessed_filtered/
  datasets:
    vbf: ${host.data_dir}/hh_vbf/
  train_dataset:
  - vbf
  test_dataset:
  - vbf
  train_dir: train
  val_dir: val
  test_dir: test
models:
  classification:
    model:
      _target_: vbf_tagger.models.LorentzNet.classification
      name: classification

In [3]:
from vbf_tagger.tools.data.dataloaders import VBFDataModule
from hydra import compose, initialize

with initialize(version_base=None, config_path="../config", job_name="test_app"):
    cfg = compose(config_name="main")

In [4]:
datamodule = VBFDataModule(cfg=cfg, data_type="train", debug_run=True)
datamodule.setup(stage="fit")

batch = next(iter(datamodule.train_dataloader()))
print("cand_featuress:", batch[0].shape)
print("cand_kinematics:", batch[1].shape)
print("node_mask:", batch[2].shape)
print("targets:", batch[3].shape)

There are 2,048 waveforms/events/jets in the dataset.
There are 2,048 waveforms/events/jets in the dataset.
Returning tensors with shapes: torch.Size([1024, 11, 16]) torch.Size([1024, 4, 16]) torch.Size([1024, 1, 16]) torch.Size([1024, 16])
cand_featuress: torch.Size([1, 11, 16])
cand_kinematics: torch.Size([1, 4, 16])
node_mask: torch.Size([1, 1, 16])
targets: torch.Size([1, 16])


In [7]:
from vbf_tagger.models.LorentzNet import classification
print(classification)

<class 'vbf_tagger.models.LorentzNet.classification'>
